In [2]:
# Important libaries
import pandas as pd
import nltk
from nltk.tokenize import WhitespaceTokenizer as w_tokenizer
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer("english")

In [3]:
df = pd.read_table("/kaggle/input/LIARbyYang/train.tsv", header = None)
df.columns = ["ID", "label", "statement", "subject", "speaker", "speaker_job", "state", "party", "pof_count", "false_count", "barelytrue_count", "halftrue_count", "mostlytrue_count", "context"]

In [4]:
df.head(10)


,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


# Merge label

In [5]:
merger = { 'pants-fire' : 0,
           'false' : 0,
           'barely-true': 0,
           'half-true': 0,
           'mostly-true': 1,
           'true': 1}
df["label"].replace(merger, inplace=True)
df.head(10)

,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,0,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,0,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,0,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,1,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


# Clean data

In [6]:
df['context'].replace('\.','',regex=True,inplace=True)

In [7]:
df.head(10)

,ID,label,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech
2,324.json,1,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,0,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release
7,153.json,0,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa"
8,5602.json,0,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,1,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


# Bag of Words Process on Statements

In [8]:
df['statement'].replace('\.','',regex=True,inplace=True) # get rid of all periods
df['statement'].replace(',','',regex=True,inplace=True) # get rid of all commas
df["statement_token"] = df["statement"].apply(nltk.word_tokenize) # tokenize statement

In [9]:
df['statement_token'].head(5)

0    [Says, the, Annies, List, political, group, su...
1    [When, did, the, decline, of, coal, start, ?, ...
2    [Hillary, Clinton, agrees, with, John, McCain,...
3    [Health, care, reform, legislation, is, likely...
4    [The, economic, turnaround, started, at, the, ...
Name: statement_token, dtype: object

In [10]:
df['stemmed_statement_token'] = df['statement_token'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

Vectorize tokens

In [11]:
df['stemmed_statement_token']

0        [say, the, anni, list, polit, group, support, ...
1        [when, did, the, declin, of, coal, start, ?, i...
2        [hillari, clinton, agre, with, john, mccain, `...
3        [health, care, reform, legisl, is, like, to, m...
4        [the, econom, turnaround, start, at, the, end,...
                               ...                        
10235    [there, are, a, larger, number, of, shark, att...
10236    [democrat, have, now, becom, the, parti, of, t...
10237    [say, an, altern, to, social, secur, that, ope...
10238    [on, lift, the, us, cuban, embargo, and, allow...
10239    [the, depart, of, veteran, affair, has, a, man...
Name: stemmed_statement_token, Length: 10240, dtype: object

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=lambda x: x) # need analyzer=lambda x: x to disable the analyzer, since the input is pandas series of lists.
X = vectorizer.fit_transform(df['stemmed_statement_token'])
new_features = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out()) # Get new features from countvectorizer



In [74]:
cv = CountVectorizer(analyzer=lambda x: x) # need analyzer=lambda x: x to disable the analyzer, since the input is pandas series of lists.
XX = cv.fit_transform(df['stemmed_statement_token'][0:3])
test = pd.DataFrame(XX.toarray(), columns = cv.get_feature_names_out()) # Get new features from countvectorizer


In [75]:
test


,'',(,),?,``,abort,administr,agre,anni,begin,...,support,that,the,third-trimest,to,took,vote,w,when,with
0,0,0,0,0,0,1,0,0,1,0,...,1,0,1,1,0,0,0,0,0,0
1,0,1,1,1,0,0,1,0,0,1,...,0,1,1,0,1,1,0,1,2,0
2,1,0,0,0,1,0,0,1,0,0,...,0,0,2,0,1,0,1,0,0,1


In [64]:
df['stemmed_statement_token'][0]

['say',
 'the',
 'anni',
 'list',
 'polit',
 'group',
 'support',
 'third-trimest',
 'abort',
 'on',
 'demand']

Chooes top 5 most common words each statement contains.

In [ ]:
def most_common_words_list (x):
    word_count_tups = nltk.FreqDist(x['stemmed_statement_token']).most_common(10) # Bag of words for each statement
    return [word for word, count in word_count_tups] # return the most
top10 = df.apply(most_common_words_list,  result_type='reduce', axis=1)
df[['top10(1)','top10(2)','top10(3)','top10(4)','top10(5)','top10(6)','top10(7)','top10(8)','top10(9)','top10(10)']] = pd.DataFrame(top10.tolist(), index= top10.index)

# bag of words for the whole dataset
word2count = {}
for words in df["stemmed_statement_token"]:
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1
            
# sort the word2count
sorted_word2count = sorted(word2count.items(), key=lambda x:x[1], reverse = True)



In [ ]:
df.head(10)

# Test

Show unique value for "context"

In [ ]:
df["context"].unique()

Show all rows that contain:

In [ ]:
df[df['context'].str.contains('Denver', na=False)]